# 📚 Conceitos Fundamentais no Spark e Databricks

## 🔍 1. Quality Checks
### ✅ O que é?
Quality Checks são **validações aplicadas aos dados para garantir sua integridade e qualidade**. Isso é feito antes de usar os dados para análise ou gravação em um banco de dados para evitar inconsistências.

### 📌 Por que é importante?
- Garante que os dados estejam **corretos**, **consistentes** e **completos**.
- Evita erros em análises e processos posteriores.

---

## 🔀 2. Deduplicate
### ✅ O que é?
Deduplicate é o processo de **remover registros duplicados de um DataFrame ou tabela**. É útil para evitar redundâncias e garantir que cada linha seja única.

### 📌 Por que é importante?
- Mantém a **qualidade e integridade dos dados**.
- Evita contagens erradas e cálculos incorretos.

---

## 📅 3. Window Function
### ✅ O que é?
As Window Functions permitem realizar **cálculos em grupos de linhas relacionadas**, mantendo cada linha original sem agrupar como o `groupBy()` faria.

### 📌 Por que é importante?
- Permite cálculos complexos como **ranking, médias móveis e agregações parciais**.
- Facilita operações que exigem contexto sobre outras linhas sem perder os dados originais.

---

## 📝 4. Apply Schema
### ✅ O que é?
O **Apply Schema** é o processo de definir explicitamente um esquema (`schema`) para um DataFrame, especificando **nomes de colunas e tipos de dados**.

### 📌 Por que é importante?
- **Garante consistência** no formato dos dados.
- Evita erros de inferência automática do Spark que podem ocorrer com dados sujos ou mal formatados.

---

## 📂 5. Operações: MERGE, APPEND e OVERWRITE

### 🔄 MERGE (Upsert)
#### ✅ O que é?
Combina dados de uma fonte com uma tabela existente, **atualizando registros correspondentes e inserindo novos registros**.

#### 📌 Uso Ideal:
- Dados mestres que precisam ser atualizados periodicamente.

---

### 📥 APPEND
#### ✅ O que é?
Adiciona **novos registros** a uma tabela sem modificar os existentes.

#### 📌 Uso Ideal:
- Adicionar dados incrementais (logs, eventos, transações, etc.).

---

### ♻️ OVERWRITE
#### ✅ O que é?
**Substitui completamente** os dados de uma tabela, apagando o conteúdo anterior.

#### 📌 Uso Ideal:
- Atualizações completas ou substituição de dados desatualizados.

---

## 📌 Resumo das Operações

| Operação  | Descrição                                                      | Uso Ideal                                    |
|-----------|----------------------------------------------------------------|--------------------------------------------|
| **MERGE** | Atualiza registros existentes e insere novos dados.           | Dados mestres que precisam de atualizações. |
| **APPEND**| Insere novos registros sem modificar os existentes.           | Dados incrementais (logs, eventos, etc.).  |
| **OVERWRITE**| Substitui completamente os dados de uma tabela.            | Relatórios completos ou recriação de tabelas. |


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame, window
from delta.tables import DeltaTable
from pyspark.sql.types import (StructType, StructField,
        IntegerType, StringType, DoubleType, DecimalType, TimestampType, ShortType)

In [0]:
def _validate_schema(df: DataFrame, expected_schema: StructType) -> bool:
    """
    Valida se o schema do DataFrame corresponde ao schema esperado.

    Parâmetros:
        df (DataFrame): O DataFrame a ser validado.
        expected_schema (StructType): O schema esperado.

    Retorna:
        bool: True se o schema corresponder, False caso contrário.
    """
    actual_schema = df.schema

    # Verifica se o número de campos corresponde
    if len(expected_schema.fields) != len(actual_schema.fields):
        return False

    # Verifica cada campo e tipo de dado
    for i, field in enumerate(actual_schema.fields):
        expected_field = expected_schema.fields[i]
        if field.name != expected_field.name or not isinstance(field.dataType, type(expected_field.dataType)):
            return False

    return True


In [0]:
def _upsert_silver_table(transformed_df: DataFrame, target_table: str, primary_keys: list, not_matched_by_source_action: str = None, not_matched_by_source_condition: str = None) -> None:
    """
    Realiza o upsert (update e insert) na tabela Delta da camada prata,
    suportando a evolução do esquema e construindo dinamicamente a condição de merge.

    Parâmetros:
        transformed_df (DataFrame): DataFrame contendo os dados transformados para inserção na camada prata.
        target_table (str): Nome da tabela de destino.
        primary_keys (list): Lista de chaves primárias para o merge.
        not_matched_by_source_action (str, opcional): Ação a ser tomada quando uma linha da tabela de destino não tiver correspondência na tabela de origem. Pode ser "DELETE" ou "UPDATE".
        not_matched_by_source_condition (str, opcional): Condição adicional para aplicar a ação definida em not_matched_by_source_action. -- use t.column = s.column -- t -> target / s -> source
    """
    spark.sql("USE CATALOG hive_metastore")
    spark.sql("USE DATABASE adventure_works_silver")

    if not spark.catalog.tableExists(target_table):
        transformed_df.write.format("delta").saveAsTable(target_table)
        print(f"Tabela {target_table} criada.")
        return

    
    merge_condition = " AND ".join([f"s.{key} = t.{key}" for key in primary_keys])

    delta_table = DeltaTable.forName(spark, target_table) # DeltaTable.forName -> Carrega uma tabela Delta existente

    merge_builder = delta_table.alias("t").merge(
        transformed_df.alias("s"),
        merge_condition
    )

    merge_builder = merge_builder.whenMatchedUpdateAll()

    merge_builder = merge_builder.whenNotMatchedInsertAll() # -> Adicionar a cláusula WHEN NOT MATCHED (inserir novos registros)

  # Se o parâmetro not_matched_by_source_action for "DELETE", adicionar a lógica para deletar linhas
    if not_matched_by_source_action and not_matched_by_source_action.upper() == "DELETE":
        # Obter as chaves das linhas na tabela de destino que não têm correspondência na tabela de origem
        unmatched_rows = delta_table.toDF().alias("t").join(
            transformed_df.alias("s"),
            on=[F.col(f"t.{key}") == F.col(f"s.{key}") for key in primary_keys],
            how="left_anti"
        )

        # Aplicar a condição adicional de exclusão, se fornecida
        if not_matched_by_source_condition:
            unmatched_rows = unmatched_rows.filter(not_matched_by_source_condition)

        # Executar a exclusão das linhas não correspondentes
        delta_table.alias("t").merge(
            unmatched_rows.alias("s"),
            merge_condition
        ).whenMatchedDelete().execute()

    # Executar o merge
    merge_builder.execute()
    
    print("Upsert executado com sucesso.")